# Metaprogramming in Julia

Everything in Julia is an expression that returns a value when executed. Every piece of the
program code is internally represented as an ordinary Julia data structure, also called an
expression

In this chapter, we will see how, by working on expressions, a Julia program
can transform and even generate new code. This is a very powerful characteristic, also
called homoiconicity

It inherits this property from Lisp, where code and data are just lists,
and where it is commonly referred to with the phrase: Code is data and data is code.

In homoiconic languages, code can be expressed in terms of the language syntax. This is the
case for the Lisp-like family of languages: Lisp, Scheme and, more recently, Clojure, which
use s-expressions

Julia is homoiconic, as are others such as Prolog, IO, Rebol, and Red. As
such, these are able to generate code during runtime, which can be subsequently executed.

We will explore this metaprogramming power by covering the following topics:
- Expressions and symbols
- Evaluation and interpolation
- Defining macros
- Built-in macros
- Reflection capabilities

# Expressions and symbols

An abstract syntax tree (AST) is a tree representation of the abstract syntactic structure of
source code written in a programming language.

When Julia code is parsed by its LLVM JIT
compiler, it is internally represented as an abstract syntax tree. The nodes of this tree are
simple data structures of the type expression Expr

An expression is simply an object that represents Julia code.

To make Julia see this as an expression and block its evaluation, we have to quote it, that is,
precede it by a colon (:) as in :(2 + 3)

In [1]:
:(2 + 3)

:(2 + 3)

In [2]:
typeof(ans)

Expr

In fact, the :
operator (also called the quote operator) sets out to treat its argument as data, not as code.

If this code is more than one line, enclose the lines between the quote and end keywords to
turn the code into an expression. For example, this expression just returns itself:

In [3]:
quote 
    a = 43
    b = a ^ 2
    a - b
end


quote
    #= In[3]:2 =#
    a = 43
    #= In[3]:3 =#
    b = a ^ 2
    #= In[3]:4 =#
    a - b
end

In fact, this is the same as :(a = 42; b = a^2; a - b). quote...end is just another
way to convert blocks of code into expressions.

In [4]:
:(a = 43; b=a^2; a-b)

quote
    a = 43
    #= In[4]:1 =#
    b = a ^ 2
    #= In[4]:1 =#
    a - b
end

In [5]:
a = :(a = 43; b=a^2; a-b)

quote
    a = 43
    #= In[5]:1 =#
    b = a ^ 2
    #= In[5]:1 =#
    a - b
end

We can give an expression such as this a name, such as e1 = :(2 + 3). We can ask for the
following information

In [8]:
e1 = :(2 + 3)

:(2 + 3)

In [9]:
e1.head

:call

indicating the kind of expression, which here is a
function call

In [10]:
e1.args

3-element Vector{Any}:
  :+
 2
 3

Indeed the expression 2 + 3 is, in fact, a call of the + function with the argument 2, and 3:
2 + 3 == + (2, 3) returns true.

The args argument consists of a symbol :+, and two
literal values, 2 and 3. Expressions are made up of symbols and literals. More complicated
expressions will consist of literal values, symbols, and sub- or nested expressions, which
can, in turn, be reduced to symbols and literals.

In [11]:
e2 = :(2 + a * b - c)

:((2 + a * b) - c)

In [12]:
e2.args

3-element Vector{Any}:
 :-
 :(2 + a * b)
 :c

2 consists of e2.args, which is a 3-element Array{Any,1} that contains :- and :c,
which are symbols, and :(2 + a * b), which is also an expression. This last expression,
in turn, is itself an expression with args:+, 2, and :(a * b); :(a * b) is an expression
with arguments and symbols: :*, :a, and :b.

In the context of an expression, symbols are used to indicate access to variables; they represent
the variable in the tree structure for the code.

In fact, the prevent evaluation character of
the quote operator (:) is already at work with symbols: after x = 5, x returns 5, but :x
returns :x.

The dump function presents the abstract syntax tree for its argument in a nice way. For
example, dump(:(2 + a * b - c)) returns the output, as shown in the following
screenshot:

In [13]:
dump(:(2 + a * b - c))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 2
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol a
            3: Symbol b
    3: Symbol c


# Evaluation and interpolation

With the definition of type Expr from the preceding section, we can also build expressions
directly from the constructor for Expr

In [14]:
e1 = Expr(:call,*,2,9)

:((*)(2, 9))

The result of an expression can be computed with the eval function, eval(e1)

In [15]:
eval(e1)

18

At the time an expression is constructed, not all the symbols have to
be defined, but they have to be defined at the time of evaluation, otherwise an error occurs.

In [19]:
e2 = Expr(:call, *, 9, :a)

:((*)(9, a))

In [20]:
eval(e2)

LoadError: MethodError: no method matching *(::Int64, ::Expr)
[0mClosest candidates are:
[0m  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at /opt/julia-1.7.1/share/julia/base/operators.jl:655
[0m  *(::T, [91m::T[39m) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at /opt/julia-1.7.1/share/julia/base/int.jl:88
[0m  *(::Union{Int16, Int32, Int64, Int8}, [91m::BigInt[39m) at /opt/julia-1.7.1/share/julia/base/gmp.jl:542
[0m  ...

In [21]:
a = 89

89

In [22]:
eval(e2)

801

Expressions can also change the state of the execution environment, for example, the
expression e3 = :(b = 1) assigns a value to b when evaluated, and even defines b, if it
doesn't exist already.

In [26]:
e3 = :(javid=1)

:(javid = 1)

In [27]:
eval(e3)

1

In [28]:
b

1

In [29]:
javid

1

To make writing expressions a bit simpler, we can use the $ operator to do interpolation in
expressions, as with $ in strings, and this will evaluate immediately when the expression is
made

In [30]:
a = 4; b = 2;

In [31]:
e4 = :(a + b)

:(a + b)

In [32]:
e4 = :($a + b)

:(4 + b)

; both expressions evaluate to 5. So, there are two kinds of
evaluation here:

- Expression interpolation (with $) evaluates when the expression is constructed
(at parse time)
- Quotation (with : or quote) evaluates only when the expression is passed to
eval at runtime

We now have the capability to build code programmatically. Inside a Julia program, we can
construct arbitrary code while it is running, and then evaluate this with eval. So, Julia can
generate the code from inside itself during the normal program execution.

This happens all the time in Julia and it is used, for example, to do things such as
generating bindings for external libraries, to reducing the repetitive boilerplate code needed
to bind big libraries, or generating lots of similar routines in other situations. Also, in the
field of robotics, the ability to generate another program and then run it is very useful. For
example: a chirurgical robot learns how to move by perceiving a human surgeon
demonstrating a procedure. Then, the robot generates the program code from that
perception, so that it is able to perform the procedure by itself.

One of the most powerful Julia tools emerging from what we discussed before is macros,
which exist in all the languages of the Lisp family.

Julia version 1.0 also introduces the concept of generated functions: such functions are
prefixed by the @generated macro and, instead of normal values, they return expressions.
We won't discuss this advanced concept here in this book.

# Defining macros

Macros are like
functions, but instead of values they take expressions (which can also be symbols or literals)
as input arguments.

When a macro is evaluated, the input expression is expanded, that is,
the macro returns a modified expression. This expansion occurs at parse time when the
syntax tree is being built, not when the code is actually executed

- Function: It takes the input values and returns the computed values at runtime
- Macro: It takes the input expressions and returns the modified expressions at
parse time

In other words, a macro is a custom program transformation. Macros are defined with the
keyword as follows:

In [33]:
macro mname 
end


@mname (macro with 0 methods)

It is invoked as @mname exp1 exp2 or @mname(exp1, exp2) (the @ sign distinguishes it
from a normal function call). The macro block defines a new scope. Macros allow us to
control when the code is executed.

In [36]:
macro macint(ex)
    quote 
        println("Start")
        $ex
        println("End")
    end
end


@macint (macro with 1 method)

In [37]:
@macint println("WTF")

Start
WTF
End


In [51]:
macro assert(ex)
#     :($ex ? nothing : error("Assertation Failed", $String(ex)))
    :($ex ? nothing : error("Assertion failed: ",$(string(ex))))
end


@assert (macro with 1 method)

In [52]:
a = "WTF"

"WTF"

In [58]:
@assert 1 == 42

LoadError: Assertion failed: 1 == 42

For example: @assert 1 == 1.0 returns nothing. @assert 1 == 42 returns
ERROR: Assertion failed: 1 == 42.

The macro replaces the expression with a ternary operator expression, which is
evaluated at runtime.

To examine the resulting expression, use the macroexpand
function as follows:

In [65]:
macroexpand(Main, :(@assert 1==42))

:(if 1 == 42
      Main.nothing
  else
      Main.error("Assertion failed: ", "1 == 42")
  end)

The third example mimics an unless construct, where branch is executed if the
condition test_cond is not true:

In [66]:
macro unless(test_con, branch)
    quote 
        if !$test_con
            $branch
        end
    end
end


@unless (macro with 1 method)

In [68]:
@unless 3<2  println("wTF")

wTF


In [76]:
arr = collect(4:10);


In [77]:
@unless 3 in arr println("arr does not contain 3")

arr does not contain 3


In [83]:
macroexpand(Base, :(@assert 3 in arr println("arr doesn't contain 3")))

:(if 3 in arr
      nothing
  else
      Base.throw(Base.AssertionError((Base.Main).Base.string(println("arr doesn't contain 3"))))
  end)

Unlike functions, macros inject the code directly into the namespace in which they are
called, possibly this is also in a different module than the one in which they were defined. It
is therefore important to ensure that this generated code does not clash with the code in the
module in which the macro is called.

When a macro behaves appropriately like this, it is
called a hygienic macro. The following rules are used when writing hygienic macros:

- Declare the variables used in the macro as local, so as not to conflict with the
outer variables
- Use the escape function esc to make sure that an interpolated expression is not
expanded, but instead is used literally
- Don't call eval inside a macro (because it is likely that the variables you are
evaluating don't even exist at that point)

In [89]:
macro mytimeint(ex)
    quote 
        local t1 = time()
        local ret_val = $ex
        local t2 = time()
        println("\ntime passed : $(t2-t1) s")
    end
end


@mytimeint (macro with 1 method)

In [90]:
@mytimeint println("WTF")

WTF

time passed : 7.295608520507812e-5 s


Macros are valuable tools which save you a lot of tedious work, and, with the quoting and
interpolation mechanism, they are fairly easy to create. You will see them being used
everywhere in Julia for lots of different tasks. Ultimately, they allow you to create domain-
specific languages (DSLs). To get a better idea of this concept, we suggest you experiment
with the other examples in the accompanying code file.

# Built-in macros

Needless to say, the Julia team has put macros to good use. To get help information about a
macro, enter a ? in the REPL, and type @macroname after the help> prompt

In [94]:
? macro

search: macro macroexpand @macroexpand @macroexpand1



```
macro
```

`macro` defines a method for inserting generated code into a program. A macro maps a sequence of argument expressions to a returned expression, and the resulting expression is substituted directly into the program at the point where the macro is invoked. Macros are a way to run generated code without calling [`eval`](@ref Base.eval), since the generated code instead simply becomes part of the surrounding program. Macro arguments may include expressions, literal values, and symbols. Macros can be defined for variable number of arguments (varargs), but do not accept keyword arguments. Every macro also implicitly gets passed the arguments `__source__`, which contains the line number and file name the macro is called from, and `__module__`, which is the module the macro is expanded in.

# Examples

```jldoctest
julia> macro sayhello(name)
           return :( println("Hello, ", $name, "!") )
       end
@sayhello (macro with 1 method)

julia> @sayhello "Charlie"
Hello, Charlie!

julia> macro saylots(x...)
           return :( println("Say: ", $(x...)) )
       end
@saylots (macro with 1 method)

julia> @saylots "hey " "there " "friend"
Say: hey there friend
```


# Built-in macros

# Testing

The @assert macro actually exists in the standard library. The standard version also
allows you to give your own error message, which is printed after ERROR: assertion
failed.

In [5]:
@assert 1 == 3

LoadError: AssertionError: 1 == 3

The Test library contains some useful macros to compare the numbers:

In [7]:
using Test

In [8]:
@test 1 == 3

Test Failed at In[8]:1
  Expression: 1 == 3
   Evaluated: 1 == 3


LoadError: [91mThere was an error during testing[39m

In [9]:
@test 1 == 1

Test Passed
  Expression: 1 == 1
   Evaluated: 1 == 1

@test with the ≈ operator tests whether the two numbers are approximately equal. @test
1 ≈ 1.1 returns Test Failed because they are not equal within the machine tolerance.
However, you can give the interval as the last argument within which they should be
equal: @test 1 ≈ 1.1 atol=0.2, which returns Test Passed, so 1 and 1.1 are within
0.2 from each other.

In [19]:
a = @test 1 ≈ 1.1 atol = 0.11

Test Passed
  Expression: ≈(1, 1.1, atol = 0.11)
   Evaluated: ≈(1, 1.1; atol = 0.11)

In [22]:
a.value

true

# Debugging

If you want to look up in the source code where and how a particular method is defined,
use @which.

In [23]:
arr = collect(1:10)

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [24]:
@which sort(arr)

sort(v::AbstractVector; kws...) in Base.Sort at sort.jl:770

In [25]:
sort(arr)

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

@show shows the expression and its result, which is handy for checking the embedded
results:

In [27]:
 99*10 + (@show 6 + 2)

6 + 2 = 8


998

# Benchmarking

For benchmarking purposes, we already know @time and @elapsed; @timed gives you the
@time results as a tuple:

In [ ]:
a

In [31]:
@time arr .^ 2

  0.103026 seconds (196.09 k allocations: 10.950 MiB, 16.26% gc time, 98.89% compilation time)


10-element Vector{Int64}:
   1
   4
   9
  16
  25
  36
  49
  64
  81
 100

In [32]:
@elapsed arr .^ 2

1.951e-5

In [33]:
@timed arr .^ 2

(value = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100], time = 1.5921e-5, bytes = 224, gctime = 0.0, gcstats = Base.GC_Diff(224, 0, 0, 5, 0, 0, 0, 0, 0))

If you are specifically interested in the allocated memory, use @allocated [x^2 for x
in 1:1000], which returns 8064.

In [45]:
@allocated [Int32(x^2) for x in collect(1:1000)]

2756440

If you are looking for a package, consult BenchmarkingTools. This has some macros and
also a good method for benchmarking.

# Starting a task

Tasks (refer to the Tasks section in Chapter 4, Control Flow) are independent units of code
execution. Often, we want to start executing them, and then continue executing the main
code without waiting for the task result. In other words, we want to start the task
asynchronously. This can be done with the @async macro:

In [53]:
a = @async println("WTF")
b = @async println("No")
println("What do you want")

What do you want
WTF
No


# Reflection capabilities

We saw in this chapter that code in Julia is represented by expressions that are data
structures of the Expr type. The structure of a program and its types can therefore be
explored programmatically just like any other data

This means that a running program can
dynamically discover its own properties, which is called reflection. We have already
encountered some of these macros or functions before:

- Use the @isdefined macro to check whether a variable is already declared, for
example if a is not declared, you get:

In [55]:
@isdefined k

false

- Use the typeof and InteractiveUtils.subtypes to query the type
hierarchy (refer to Chapter 6, More on Types, Methods, and Modules)

In [56]:
typeof(a)

Task

- Use the methods(f) to see all the methods of a function f (refer to Chapter 3,
Functions)

In [58]:
methods(a)

# 0 methods:

- names and types: given a type Person:

In [59]:
struct Person 
    name::String
    age::Float32
end


In [61]:
fieldnames(Person)

(:name, :age)

In [62]:
Person.types

svec(String, Float32)

- To inspect how a function is represented internally, you can use code_lowered:

In [64]:
code_lowered(^, (Int64, Int64))

1-element Vector{Core.CodeInfo}:
 CodeInfo(
1 ─ %1 = Base.power_by_squaring(x, p)
└──      return %1
)

Or you can use code_typed to see the type-inferred form:

In [65]:
code_typed(^, (Int64, Int64))

1-element Vector{Any}:
 CodeInfo(
1 ─ %1 = invoke Base.power_by_squaring(x::Int64, p::Int64)::Int64
└──      return %1
) => Int64

Using code_typed can show you whether your code is type-optimized
for performance: if the Any type is used instead of an appropriate specific
type that you would expect, then the type annotation in your code can
certainly be improved, leading most likely to speeding up the program's
execution

- To inspect the code generated by the LLVM engine, use code_llvm, and, to see
the assembly code generated, use code_native (refer to the How Julia works
section in Chapter 1, Installing the Julia Platform).

In [67]:
code_llvm(^, (Int64, Int64))

;  @ intfuncs.jl:290 within `^`
define i64 @"julia_^_3521"(i64 signext %0, i64 signext %1) #0 {
top:
  %2 = call i64 @j_power_by_squaring_3523(i64 signext %0, i64 signext %1) #0
  ret i64 %2
}


In [69]:
code_native(^, (Int64, Int64))

	.text
; ┌ @ intfuncs.jl:290 within `^`
	subq	$8, %rsp
	movabsq	$power_by_squaring, %rax
	callq	*%rax
	popq	%rcx
	retq
	nopw	%cs:(%rax,%rax)
; └


While reflection is not necessary for many of the programs that you will write, it is very
useful for IDEs to be able to inspect the internals of an object, as well as for tools generating
automatic documentation, and for profiling tools. In other words, reflection is
indispensable for tools that need to inspect the internals of code objects programmatically.
You should also look at the MacroTools package (from Mike Innes) which has some good
examples of macros.